# Master Notebook

This is the noebook that will contain all of the pieces woven together and can be run all at once

# Reading the video

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from base64 import b64encode

from moviepy.editor import *

In [2]:
input_path = 'input/input_with_audio.mp4'

In [3]:
# use moviepy to find the aspect ration of this .mp4 file

# USER INPUT
clip = VideoFileClip(input_path)
aspect_ratio = clip.size
print("Clip size ", aspect_ratio)

height = clip.size[1]
width = clip.size[0]

duration = clip.duration
print("Duration ", duration)

Clip size  [384, 864]
Duration  30.17


In [4]:
# Display the video
mp4 = open(input_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [5]:
# extract the audio from this file
audio = clip.audio
audio.write_audiofile(f'audio/audio_to_transcribe.mp3')

MoviePy - Writing audio in audio/audio_to_transcribe.mp3


chunk:  11%|█         | 74/666 [00:00<00:01, 393.01it/s, now=None]

MoviePy - Done.


In [6]:
# remove the audio from the file
clip = clip.without_audio()
clip.write_videofile(f'video/step1.mp4')

# NOTE step1.mp4 has no audio

Moviepy - Building video video/step1.mp4.
Moviepy - Writing video video/step1.mp4



t:   3%|▎         | 26/906 [00:00<00:06, 139.06it/s, now=None]

Moviepy - Done !
Moviepy - video ready video/step1.mp4


## Transcribe the text using whispr

In [7]:
import whisper
import os

audio_input_path = "audio/audio_to_transcribe.mp3" 

model = whisper.load_model("base.en")

In [8]:
result = model.transcribe(audio_input_path)
print(result["text"])

 Today we are going to be looking at how to use the VITN application to view your exam schedule. We will begin by opening the VITN application. Once it is opened up, we tap on the refresh button in order to get the latest data and keep it updated. Then we open the sidebar by tapping on the sidebar icon, go into academics, down and exam schedule. From here we can select our exam and now view the exam venue, see details, time and date. Thank you.


In [9]:
transcript = result

In [10]:
print(result)

{'text': ' Today we are going to be looking at how to use the VITN application to view your exam schedule. We will begin by opening the VITN application. Once it is opened up, we tap on the refresh button in order to get the latest data and keep it updated. Then we open the sidebar by tapping on the sidebar icon, go into academics, down and exam schedule. From here we can select our exam and now view the exam venue, see details, time and date. Thank you.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.8000000000000003, 'text': ' Today we are going to be looking at how to use the VITN application to view your exam', 'tokens': [50363, 6288, 356, 389, 1016, 284, 307, 2045, 379, 703, 284, 779, 262, 569, 2043, 45, 3586, 284, 1570, 534, 2814, 50553], 'temperature': 0.0, 'avg_logprob': -0.22808042346921742, 'compression_ratio': 1.7233201581027668, 'no_speech_prob': 0.13063941895961761}, {'id': 1, 'seek': 0, 'start': 3.8000000000000003, 'end': 4.8, 'text': ' schedule.', 'tokens': [5

## Text Generation

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, TransformChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [12]:
# Loading my OPENAI KEY as an environment variable
from dotenv import load_dotenv
load_dotenv()

True

### Steps Generation

In [13]:
# Prompt template to generate the steps text
topic_template = """
You are given the transcript of a video of me using an app.
You are to convert the given text of actions performed in the video into an organized list of steps and properly number them.
You are to also timestamp each of the steps as per the transcript. 
If the app name does'nt match the transcript, use the name given in the app description.
The output must be a python list of tuples like [(step_text, timestamp), (step_text, timestamp)] where each step_text is a step and timestamp is the start of the text in seconds . Be informative and do not make up things.

App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```

The Transcript:
```
{transcript}
```
Do not give any additional text. No talk, just go. 
"""

In [14]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are an expert at making tutorial videos and very good at defining tasks in simple terms.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc","tutorial_desc", "transcript"]))

In [15]:
# Chain to write the create the steps from the input text
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
steps_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='steps')

In [16]:
# Description of the app we are using

# USER INPUT
app_desc = """VITian is an application built for students of VIT to access their student data. 
We can use the app to find out examination allotments."""

# Tutorial Description

#USER INPUT
tutorial_desc = """This is a video tutorial on how to use the VITian app to check exam schedule and timings."""

# transcript has been acquired from the whisper module

In [17]:
# testing the steps chain + prompt engineering
op = steps_chain(inputs={"app_desc": app_desc, "tutorial_desc": tutorial_desc, "transcript": transcript}, return_only_outputs=True)

In [18]:
print(op)

{'steps': 'Here is the list of steps with timestamps:\n\n```python\n[\n    ("Open the VITian application", 4.8),\n    ("Tap on the refresh button to get the latest data", 9.08),\n    ("Open the sidebar by tapping on the sidebar icon", 15.2),\n    ("Go into academics, then exam schedule", 21.04),\n    ("Select your exam to view the exam venue, details, time and date", 23.4)\n]\n```'}


In [19]:
print(op['steps'])
steps = op['steps']

Here is the list of steps with timestamps:

```python
[
    ("Open the VITian application", 4.8),
    ("Tap on the refresh button to get the latest data", 9.08),
    ("Open the sidebar by tapping on the sidebar icon", 15.2),
    ("Go into academics, then exam schedule", 21.04),
    ("Select your exam to view the exam venue, details, time and date", 23.4)
]
```


In [20]:
# steps = """Here is the list of steps with timestamps:

# ```python
# [
#     ("Open the VITN application", 4.8),
#     ("Tap on the refresh button to get the latest data", 9.08),
#     ("Open the sidebar by tapping on the sidebar icon", 15.2),
#     ("Go into academics, then exam schedule", 21.04),
#     ("Select your exam to view the exam venue, details, time and date", 23.4)
# ]
# ```"""

In [21]:
import re

tuples = re.findall(r'\(([^)]+)', steps)
# array of timestamps of each step
timers = [0]

step_list = []

for i in tuples:
    text = re.findall(r'[^"]+', i)
    # print(text[0])
    timer = float(re.findall(r'\d+\.\d+', i)[0])
    # print(timer)
    print((text[0],timer))
    step_list.append((text[0],timer))
    timers.append(timer)
    print("=========")

# we want the starting timestamps, not the end
timers.pop()

('Open the VITian application', 4.8)
('Tap on the refresh button to get the latest data', 9.08)
('Open the sidebar by tapping on the sidebar icon', 15.2)
('Go into academics, then exam schedule', 21.04)
('Select your exam to view the exam venue, details, time and date', 23.4)


23.4

In [22]:
step_list

[('Open the VITian application', 4.8),
 ('Tap on the refresh button to get the latest data', 9.08),
 ('Open the sidebar by tapping on the sidebar icon', 15.2),
 ('Go into academics, then exam schedule', 21.04),
 ('Select your exam to view the exam venue, details, time and date', 23.4)]

In [23]:
timers

[0, 4.8, 9.08, 15.2, 21.04]

### Narrator Script Generation

#### Intro and Outro

In [24]:
# Prompt template to generate the intro and outro text
topic_template = """
You are giving the intro and outro to a tutorial video.
You are given a description about the app as well as the tutorial topic.
Generate text for the narrator to read out to the viewer and nothing more.
Finally, Generate an appropriate short title for the video.
Output format:
```
intro_text, outro_text, title_text
```

App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```
"""

In [25]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are a narrator with experience in making tech tutorial videos.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc", "tutorial_desc"]))

In [26]:
# Chain to write the script for the narrator
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
intro_outro_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='text')

In [27]:
op3 = intro_outro_chain(inputs={"app_desc": app_desc, "tutorial_desc": tutorial_desc, "steps": steps}, return_only_outputs=True)

In [28]:
# use regex to select text inside double quotes
intro_outro = re.findall(r'"([^"]*)"', op3['text'])
for i in intro_outro:
    print(i)
    print("===================================")

    # intro_outro [0] -> intro
    # intro_outro [1] -> outro
    # intro_outro [2] -> title

Hello everyone, welcome to our tutorial video. Today, we will be exploring the VITian app, a dedicated application designed specifically for students of VIT. This app is a one-stop solution for accessing all your student data. In this tutorial, we will guide you on how to use the VITian app to check your exam schedule and timings. So, let's dive in and make the most out of this amazing app.
Thank you for watching this tutorial. We hope you now have a clear understanding of how to use the VITian app to check your exam schedule and timings. Remember, this app is a great tool to keep track of all your student data. Don't forget to explore other features of the VITian app. If you found this video helpful, please like, share, and subscribe to our channel for more such tutorials. Until next time, happy studying!
Using VITian App: A Guide to Check Your Exam Schedule and Timings


#### Main Script 

In [29]:
# Prompt template to generate teh narrator text
topic_template = """
You are to narrate a tutorial video about using an app. Be informative and do not make up things.
You are given a description about the app as well as the tutorial topic.
You will then be given the list of steps taken in the video alongside the timestamps where they start.
Your narration must abide by the timestamps. You can make a maximum of 3 words per second in the time window for each step.
Generate text for the narrator to read out to the viewer and nothing more.
Output must be a python list of strings where each string is the text for each step.
Do NOT give the user a greeting at the start or end of the narartion.
Output format:
```
[step_1_text, step_2_text, step_3_text, step_4_text]
```

App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```

The steps(with timestamps) being performed in the video are:
```
{steps}
```

Output example:
```
["We begin by opening up the application.", "Then, tap on the GET OTP button", "Now, enter your OTP from your email", "Enter your name and phone number to get registered.", "Now, you can use your account."]
```

"""

In [30]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are a narrator with experience in making tech tutorial videos.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc", "tutorial_desc", "steps"]))

In [31]:
# Chain to write the script for the narrator
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
script_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='script')

In [32]:
op2 = script_chain(inputs={"app_desc": app_desc, "tutorial_desc": tutorial_desc, "steps": steps}, return_only_outputs=True)

In [33]:
print(op2['script'])
script = op2['script']

["Start by opening the VITian application.", "Next, tap the refresh button for the latest data.", "Then, open the sidebar by tapping its icon.", "Navigate to academics, then exam schedule.", "Finally, select your exam to view its details."]


In [34]:
# extract strings inside double quotes
strings = re.findall(r'"([^"]*)"', script)

script = strings

for i in script:
    print(i)
    print("======================")

Start by opening the VITian application.
Next, tap the refresh button for the latest data.
Then, open the sidebar by tapping its icon.
Navigate to academics, then exam schedule.
Finally, select your exam to view its details.


## Audio Generation

In [35]:
# # generate audio from text
# text_prompt = """
#      Hello, my name is Suno. And, uh — and I like pizza. [laughs] 
#      But I also have other interests such as playing tic tac toe.
# """
# # Male Speaker
# audio_array = generate_audio(text_prompt, history_prompt="v2/en_speaker_6")

# # # Female Speaker
# # audio_array = generate_audio(text_prompt, history_prompt="v2/en_speaker_6")
# Audio(audio_array, rate=SAMPLE_RATE) 

### pyttsx3

### pyttsx3

In [ ]:
import pyttsx3
engine = pyttsx3.init()
# engine.say("Speech Testing")
# engine.runAndWait()

In [ ]:
j = 0
audio_file_paths = []
for i in script:
    # print the message being said
    print(i)
    # print the time the message starts
    output_path = "audio/Audio_Step_" + str(j) + ".mp3"
    engine.save_to_file(i, output_path)
    engine.runAndWait()    
    audio_file_paths.append(output_path)
    j+=1    
    print("===========================================")

Start by opening the VITian application.
Next, tap the refresh button for the latest data.
Then, open the sidebar by tapping its icon.
Navigate to academics, then exam schedule.
Finally, select your exam to view its details.


### gTTS

In [36]:
import pyttsx3
engine = pyttsx3.init()
# engine.say("Speech Testing")
# engine.runAndWait()

In [37]:
j = 0
audio_file_paths = []
for i in script:
    # print the message being said
    print(i)
    # print the time the message starts
    output_path = "audio/Audio_Step_" + str(j) + ".mp3"
    engine.save_to_file(i, output_path)
    engine.runAndWait()    
    audio_file_paths.append(output_path)
    j+=1    
    print("===========================================")

Start by opening the VITian application.
Next, tap the refresh button for the latest data.
Then, open the sidebar by tapping its icon.
Navigate to academics, then exam schedule.
Finally, select your exam to view its details.


#### Generating audio for the intro and outro

In [38]:
intro_outro[0]

"Hello everyone, welcome to our tutorial video. Today, we will be exploring the VITian app, a dedicated application designed specifically for students of VIT. This app is a one-stop solution for accessing all your student data. In this tutorial, we will guide you on how to use the VITian app to check your exam schedule and timings. So, let's dive in and make the most out of this amazing app."

In [39]:
# Generate intro audio
engine.save_to_file(intro_outro[0], "audio/intro.mp3")
engine.runAndWait()

# Generate outro audio
engine.save_to_file(intro_outro[1], "audio/outro.mp3")
engine.runAndWait()

In [40]:
# audio_file_paths

## Using pydub to merge into a singular audio file with appropriate timings

In [41]:
from pydub import AudioSegment

In [42]:
from pydub import AudioSegment

def merge_audio_with_silence(filenames, start_times):
    # Load the first audio file
    combined_audio = AudioSegment.from_file(filenames[0])
    
    # Add the required silence before the first audio file
    silence_duration = start_times[0] * 1000  # Convert seconds to milliseconds
    combined_audio = AudioSegment.silent(duration=silence_duration) + combined_audio
    
    # For each subsequent audio file...
    for i in range(1, len(filenames)):
        # Load the audio file
        audio = AudioSegment.from_file(filenames[i])
        # print("Now reading " + filenames[i])
        
        # Calculate the required silence duration
        len_so_far = len(combined_audio) / 1000.0
        silence_duration = (start_times[i] - len_so_far) * 1000  # Convert seconds to milliseconds
        # print("len_so_far (in seconds): " , len_so_far)
        # print("we gotta start at (in seconds): ", start_times[i])
        # print("silence_duration (in milliseconds): " , silence_duration)
        if silence_duration > 0:
            silence = AudioSegment.silent(duration=silence_duration)
        else:
            silence = AudioSegment.empty()
        
        # Append the silence and the audio to the combined audio
        combined_audio += silence + audio
        # print("============================================================")
    
    return combined_audio


In [43]:
print(timers)
print(audio_file_paths)

[0, 4.8, 9.08, 15.2, 21.04]
['audio/Audio_Step_0.mp3', 'audio/Audio_Step_1.mp3', 'audio/Audio_Step_2.mp3', 'audio/Audio_Step_3.mp3', 'audio/Audio_Step_4.mp3']


In [44]:
# Merge the audio files
merged_audio = merge_audio_with_silence(audio_file_paths, timers)

# Save the merged audio to a file
output_path = "audio/merged_audio.mp3"
merged_audio.export(output_path)

<_io.BufferedRandom name='audio/merged_audio.mp3'>

## We have successfully generated the narrator audio and generated the step texts.

TODO: Overlay the audio onto out final video file <br>
Use moviepy to display the steps text on the video

# Video Editing 
## Modify the aspect ratio to 4:3
(Can be 16 : 9 also)

In [45]:
# Calculate desired width for a 4:3 aspect ratio
desired_width_4_3 = int(height * 4 / 3)

# Calculate the width of black bars to add to each side
black_bar_width = (desired_width_4_3 - width) // 2

desired_width_4_3, black_bar_width

(1152, 384)

In [46]:
# For a 4:3 aspect ratio with the given height, the desired width is 1152 pixels. 
# This means we'll need to add black bars, each of width 384 pixels, to both sides of the video.

In [47]:
# generate a jpeg that is just black pixels
black_image = np.zeros((height, desired_width_4_3, 3), dtype=np.uint8)
black_image[:] = (0, 0, 0)
plt.imsave('image_background/black_background.jpeg', black_image)

# We can give user the choice of color here

In [48]:
black_image = ImageClip("image_background/black_background.jpeg", duration = duration)

In [49]:
step2_vid = CompositeVideoClip([black_image, clip.set_position("center")])

In [50]:
# save the video
step2_vid.write_videofile("video/step2.mp4")

# NOTE step2.mp4 has the black bars added to make it 4:# ratio

Moviepy - Building video video/step2.mp4.
Moviepy - Writing video video/step2.mp4



Moviepy - Done !
Moviepy - video ready video/step2.mp4


### Overlay the narrator audio onto the video

In [51]:
muted_video = VideoFileClip("video/step2.mp4")
narrator_audio = AudioFileClip("audio/merged_audio.mp3")

In [52]:
video_with_new_audio = step2_vid.set_audio(narrator_audio)

In [53]:
output_path = "video/step3.mp4"
video_with_new_audio.write_videofile(output_path, audio_codec='aac')

Moviepy - Building video video/step3.mp4.
MoviePy - Writing audio in step3TEMP_MPY_wvf_snd.mp4


chunk:   0%|          | 0/535 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video video/step3.mp4



Moviepy - Done !
Moviepy - video ready video/step3.mp4


### Generate text images that we can overlay onto the video for the steps text

In [54]:
video = VideoFileClip("video/step3.mp4")
annotations = step_list

In [55]:
print(timers)
step_list

[0, 4.8, 9.08, 15.2, 21.04]


[('Open the VITian application', 4.8),
 ('Tap on the refresh button to get the latest data', 9.08),
 ('Open the sidebar by tapping on the sidebar icon', 15.2),
 ('Go into academics, then exam schedule', 21.04),
 ('Select your exam to view the exam venue, details, time and date', 23.4)]

In [56]:
from PIL import Image, ImageDraw, ImageFont

In [57]:
# Directory to store the generated images
image_dir = "image/text_images"

# List to store paths of generated images
image_paths = []

In [58]:
# Since the text must be drawn in the black border sections, it may cause overfill.
# So we need to implement word wrap

def get_wrapped_text(text, font, line_length):
        lines = ['']
        for word in text.split():
            line = f'{lines[-1]} {word}'.strip()
            if font.getlength(line) <= line_length:
                lines[-1] = line
            else:
                lines.append(word)
        return '\n'.join(lines)

In [59]:
# Function to generate images for each text

# NOTE: We can edit this later to make the text boxes look better and update the fonts, etc.
# NOTE: We can also add a background color to the text boxes
def generate_text_image(text, fontsize=24, color="white"):
    
    # Create a blank image with transparent background    
    img = Image.new('RGBA', (black_bar_width-4, 240), (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)
    
    # Use a truetype font
    font_path = "./font/Lato/Lato-Black.ttf"
    font = ImageFont.truetype(font_path, fontsize)

    #  get the wrapped text
    wrapped_text = get_wrapped_text(text, font, black_bar_width)
    
    # Calculate text width and height
    # text_width, text_height = draw.textsize(text, font)
    text_width, text_height = draw.multiline_textsize(wrapped_text, font=font)
    
    # Calculate X, Y position of the text
    x = (img.width - text_width) // 2
    y = (img.height - text_height) // 2
    
    # Draw the text onto the image
    draw.text((x, y), wrapped_text, font=font, fill=color)
    

    # Save the image and return the path
    image_path = image_dir + '/' + text.replace(' ', '_') + ".png"
    # display the image
    # img.show()
    img.save(image_path)
    return image_path

In [60]:
# Generate images for each text in annotations
for text, _ in annotations:
    image_paths.append(generate_text_image(text))

### Create the intro video

In [61]:
# Create the intro image

# Create a blank image with transparent background    
intro_img = Image.new('RGBA', (desired_width_4_3, height), (255, 255, 255, 0))
draw = ImageDraw.Draw(intro_img)

# Use a truetype font
font_path = "./font/Lato/Lato-Black.ttf"
font = ImageFont.truetype(font_path, 30)

#  get the wrapped text
wrapped_text = get_wrapped_text(intro_outro[2], font, desired_width_4_3//2)

# Calculate text width and height
# text_width, text_height = draw.textsize(text, font)
text_width, text_height = draw.multiline_textsize(wrapped_text, font=font)

# Calculate X, Y position of the text
x = (intro_img.width - text_width) // 2
y = (intro_img.height - text_height) // 2

# Draw the text onto the image
draw.text((x, y), wrapped_text, font=font, fill='black')

# Save the image and return the path
image_path = image_dir + "/intro_img.png"
# display the image
# img.show()
intro_img.save(image_path)

In [62]:
intro_audio = AudioFileClip("audio/intro.mp3")
intro_length = intro_audio.duration

In [63]:
intro_length

21.86

In [64]:
image = ImageClip(image_path).set_duration(intro_length)

# Set the audio of the image clip to be the audio file
video = image.set_audio(intro_audio)

# Write the result to a video file
video.write_videofile("video/intro_video.mp4", fps=24)

Moviepy - Building video video/intro_video.mp4.
MoviePy - Writing audio in intro_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video/intro_video.mp4



Moviepy - Done !
Moviepy - video ready video/intro_video.mp4


### Create the outro video

In [65]:
# Create the intro image

# Create a blank image with transparent background    
outro_img = Image.new('RGBA', (desired_width_4_3, height), (255, 255, 255, 0))
draw = ImageDraw.Draw(outro_img)

# Use a truetype font
font_path = "./font/Lato/Lato-Black.ttf"
font = ImageFont.truetype(font_path, 30)

#  get the wrapped text
wrapped_text = get_wrapped_text("Thanks for watching!", font, desired_width_4_3//2)
# print(wrapped_text)

# Calculate text width and height
# text_width, text_height = draw.textsize(text, font)
text_width, text_height = draw.multiline_textsize(wrapped_text, font=font)

# Calculate X, Y position of the text
x = (outro_img.width - text_width) // 2
y = (outro_img.height - text_height) // 2

# Draw the text onto the image
draw.text((x, y), wrapped_text, font=font, fill='black')

# Save the image and return the path
image_path = image_dir + "/outro_img.png"
# display the image
# img.show()
outro_img.save(image_path)

In [66]:
outro_audio = AudioFileClip("audio/outro.mp3")
outro_length = outro_audio.duration

In [67]:
outro_length

23.33

In [68]:
image = ImageClip(image_path).set_duration(outro_audio.duration)

# Set the audio of the image clip to be the audio file
video = image.set_audio(outro_audio)

# Write the result to a video file
video.write_videofile("video/outro_video.mp4", fps=24)

Moviepy - Building video video/outro_video.mp4.
MoviePy - Writing audio in outro_videoTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 0/515 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video video/outro_video.mp4



Moviepy - Done !
Moviepy - video ready video/outro_video.mp4


### Overlay the text images onto the video

In [69]:
annotations

[('Open the VITian application', 4.8),
 ('Tap on the refresh button to get the latest data', 9.08),
 ('Open the sidebar by tapping on the sidebar icon', 15.2),
 ('Go into academics, then exam schedule', 21.04),
 ('Select your exam to view the exam venue, details, time and date', 23.4)]

In [70]:
video = VideoFileClip("video/step3.mp4")
annotations = step_list
print(timers)
step_list

[0, 4.8, 9.08, 15.2, 21.04]


[('Open the VITian application', 4.8),
 ('Tap on the refresh button to get the latest data', 9.08),
 ('Open the sidebar by tapping on the sidebar icon', 15.2),
 ('Go into academics, then exam schedule', 21.04),
 ('Select your exam to view the exam venue, details, time and date', 23.4)]

In [71]:
# Timers has the start times of each stp and annotations has the end times of each step.

In [72]:
i = 0
all_clips = [video]
for path in image_paths:
    start_time = timers[i]
    
    end_time = annotations[i][1]

    if(i == len(timers) - 1):
        end_time += 2
    # if it is the last clip, add a few seconds as the video will have silence in the end.
    
    # print(start_time, end_time)
    text = ImageClip(path).set_start(start_time).set_end(end_time).set_position(("left", 0), relative = True)
    all_clips.append(text)
    i+=1

mid_vid = CompositeVideoClip(all_clips)      
mid_vid.write_videofile("video/step4.mp4")

Moviepy - Building video video/step4.mp4.
MoviePy - Writing audio in step4TEMP_MPY_wvf_snd.mp3


chunk:  25%|██▌       | 169/666 [00:00<00:00, 910.06it/s, now=None]

MoviePy - Done.
Moviepy - Writing video video/step4.mp4



Moviepy - Done !
Moviepy - video ready video/step4.mp4


In [73]:
intro_video = VideoFileClip("video/intro_video.mp4")
mid_vid = VideoFileClip("video/step4.mp4")
outro_video = VideoFileClip("video/outro_video.mp4")

In [74]:
final_video = concatenate_videoclips([intro_video, mid_vid, outro_video])

# Write the result to a file
final_video.write_videofile("final_video.mp4", codec="libx264")


Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 0/1666 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video final_video.mp4



t:  85%|████████▍ | 1917/2266 [00:26<00:04, 77.40it/s, now=None]